## 예스24티켓 크롤링
#### 공연 개별 상세페이지에서 정보 가져오기

In [4]:
from selenium import webdriver
import time

path="C:/chromedriver.exe"
driver=webdriver.Chrome(path)

In [5]:
def get_data(p_type):
    
    p_info={'Type':'', 'Title':'', 'TicketOpen':'', 'Venue':'', 'StartDate':'', 'EndDate':'', 'Time':'',
           'RunningTime':'', 'Price':'', 'BuyTicket':'예스24', 'PosterSrc':'', 'RegisteredTime':'', 'Performer':''}
    performer=[]
    
#     p_info={'Type':'', 'Title':'', 'Subtitle':'', 'Venue':'', 'StartDate':'', 'EndDate':'', 'Performer':''}
#     performer=[]
    
    # Type
    p_info['Type']=p_type
    
    # Title
    p_info['Title']=driver.find_element_by_class_name('rn-big-title').text
    
    # RunningTime
    div=driver.find_element_by_class_name('rn-product-area1')
    dd=div.find_elements_by_tag_name('dd')
    p_info['RunningTime']=dd[1].text
    p_info['RunningTime']=p_info['RunningTime'][1:]
    
    # PosterSrc
    try:
        p_info['PosterSrc']=driver.find_element_by_xpath('//*[@id="mainForm"]/div[10]/div/div[1]/div[1]/div[1]/img').get_attribute('src')
    except:
        p_info['PosterSrc']=driver.find_element_by_xpath('//*[@id="mainForm"]/div[9]/div/div[1]/div[1]/div/img').get_attribute('src')
    
    # Price
    dd = driver.find_element_by_class_name('rn-product-price')
    try:
        for li in dd.find_elements_by_tag_name('li'):
            p_info['Price']=p_info['Price']+li.text+', '
        p_info['Price']=p_info['Price'][:-2]
    except IndexError:
        pass
    
    # Date
    p_info['StartDate']=driver.find_element_by_class_name('ps-date').text.split(' ', 2)[0]
    p_info['EndDate']=driver.find_element_by_class_name('ps-date').text.split(' ', 2)[2]
    
    # Time
    try:
        p_info['Time']=driver.find_element_by_xpath('//*[@id="mainForm"]/div[10]/div/div[1]/div[2]/div[2]/dl/dd[1]').text
    except:
        p_info['Time']=driver.find_element_by_xpath('//*[@id="mainForm"]/div[9]/div/div[1]/div[2]/div[2]/dl/dd[1]').text
    if(p_info['Time'].find('시')==-1):
        p_info['Time']=' '
    p_info['Time'].replace('\n', ' ')
    
    # Venue
    try:
        p_info['Venue']=driver.find_element_by_class_name('ps-location').text
    except:
        p_info['Venue']=driver.find_element_by_class_name('ps-location2').text
    
    # Performer
    for i in driver.find_elements_by_class_name('rn-product-peole'):
        performer.append(i.text)
    p_info['Performer']=performer
        
    print(p_info)
    
    return p_info

In [6]:
def get_link():
    
    link=[]
    
    # 무한 스크롤
    last_height=driver.execute_script("return document.body.scrollHeight;")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(0.5)
        # Calculate new scroll height and compare with last scroll height
        new_height=driver.execute_script("return document.body.scrollHeight")
        if new_height==last_height:
            break
        last_height=new_height
    
    # 상세페이지로 접근하는 웹요소를 찾아 url에 입력할 번호만 파싱
    for row in driver.find_elements_by_class_name('ms-list-imgs'):
        for col in row.find_elements_by_tag_name('a'):
            s=col.get_attribute('onclick')
            s1=s.split('(')[1]
            s2=s1.split(')')[0]
            link.append(s2)
    
    return link

In [ ]:
p_type={'15456':'콘서트', '15457':'뮤지컬', '15458':'연극', '15459':'클래식', '15460':'전시'}
p_data=[]       # 공연정보 리스트

for key in p_type:
    
    # [장르 - 전체보기 - 신상품순] 페이지 접근
    driver.get("http://ticket.yes24.com/New/Genre/GenreList.aspx?genretype=1&genre="+key)
    time.sleep(0.5)
    driver.find_element_by_xpath('/html/body/section/p[1]/span[2]/a').click()
    
    # 각 공연의 상세페이지로 접근하기 위해 클릭할 웹요소 p_link에 가져오기
    p_link=get_link()
    
    # p_link의 주소로 상세페이지에 접근하여 데이터 가져오기
    for link in p_link:
        
        driver.get('http://ticket.yes24.com/Perf/'+link)
        time.sleep(0.5)
            
        # 상세페이지에서 얻은 공연정보 데이터를 p_data에 추가하기
        p_data.append(get_data(p_type[key]))
        

{'Type': '콘서트', 'Title': 'ATEEZ（에이티즈）OFFICIAL FANCLUB 〈ATINY〉2기 모집', 'TicketOpen': '', 'Venue': '회원모집', 'StartDate': '2021.01.31', 'EndDate': '2021.01.31', 'Time': ' ', 'RunningTime': '--', 'Price': '팬클럽가입 30,000원', 'BuyTicket': '예스24', 'PosterSrc': 'http://tkfile.yes24.com/upload2/PerfBlog/202012/20201208/20201208-38148.jpg', 'RegisteredTime': '', 'Performer': ['ATEEZ（에이티즈）']}
{'Type': '콘서트', 'Title': 'ATEEZ（에이티즈）OFFICIAL FANCLUB 〈ATINY〉2기 모집 [1기전용]', 'TicketOpen': '', 'Venue': '회원모집', 'StartDate': '2021.01.31', 'EndDate': '2021.01.31', 'Time': ' ', 'RunningTime': '--', 'Price': '팬클럽가입 30,000원', 'BuyTicket': '예스24', 'PosterSrc': 'http://tkfile.yes24.com/upload2/PerfBlog/202012/20201208/20201208-38147.jpg', 'RegisteredTime': '', 'Performer': ['ATEEZ（에이티즈）']}
{'Type': '콘서트', 'Title': '2020-21 조명섭 단독콘서트 [달밤음악회] - 앵콜', 'TicketOpen': '', 'Venue': '경희대학교 평화의전당', 'StartDate': '2021.01.30', 'EndDate': '2021.01.31', 'Time': '2021년 1월 30일(토) 오후 6시\n2021년 1월 31일(일) 오후 3시', 'RunningTime': '120분',

In [ ]:
## MySql DB에 데이터 넣기

import pymysql

conn=pymysql.connect(host='localhost', user='root', password='asdf1234', db='ticket_test', charset='utf8mb4')
try:
    with conn.cursor() as cursor:
        sql='INSERT ignore INTO yes24 (type, title, subtitle, venue, start_date, end_date) VALUES (%s, %s, %s, %s, %s, %s)'
        for r in p_data:
#             if r[end_Date]=='오픈런':
#                 cursor.execute(sql, (r[1], r[3], '', r[0]))
#             else:
            cursor.execute(sql, (r['Type'], r['Title'], r['Subtitle'], r['Venue'], r['open_Date'], r['end_Date'],))
            conn.commit()
finally:
    conn.close()